### DATA PREPARATION

Importing necssary libraries

In [ ]:
import pandas as pd
import numpy as np

import df_helper as dfh

df_helper = dfh.df_helper()

Loading the data

In [ ]:

admissions = pd.read_csv('data/admissions.csv')
omr = pd.read_csv('data/omr.csv')
diagnoses = pd.read_csv('data/diagnoses_icd.csv')
patients=pd.read_csv("data/patients.csv")
diagnoses2=pd.read_csv("data/d_icd_diagnoses.csv")
prescriptions=pd.read_csv("data/prescriptions.csv")

The first step was to view the data.

In [ ]:
admissions.head()

In [ ]:
omr.head()

In [ ]:
patients.head()

In [ ]:
prescriptions.head()

In [ ]:
diagnoses.head()

In [ ]:
diagnoses2.head()

In [ ]:
# Function to split blood pressure and convert to numeric
def preprocess_blood_pressure(bp_str):
    if pd.isna(bp_str) or not isinstance(bp_str, str):
        return pd.Series([None, None])
    try:
        systolic, diastolic = map(int, bp_str.split('/'))
        return pd.Series([systolic, diastolic])
    except:
        return pd.Series([None, None])

In [ ]:
pivot_omr = omr.pivot_table(
    index=['subject_id'], 
    columns='result_name', 
    values='result_value', aggfunc="last"
).reset_index()

df_helper.drop_columns(pivot_omr,[
    "Blood Pressure Lying", 
    "Blood Pressure Sitting", 
    "Blood Pressure Standing (1 min)", 
    "Blood Pressure Standing (3 mins)"
])

df_helper.rename_columns(pivot_omr, {
    "BMI (kg/m2)":"bmi",
    "Blood Pressure":"blood pressure",
    "Height (Inches)":"height",
    "Weight (Lbs)":"weight"
})

pivot_omr = df_helper.convert_inches_to_meters(pivot_omr, "height", 2)
pivot_omr = df_helper.convert_lbs_to_kg(pivot_omr, "weight", 2)

# Apply the function and expand into separate columns
pivot_omr[['bp_systolic', 'bp_diastolic']] = pivot_omr['blood pressure'].apply(preprocess_blood_pressure)
df_helper.drop_columns(
    pivot_omr,[
        "blood pressure",
    ]
)

pivot_omr

In [ ]:
#defining t a function to define the base of all dataframes
base_df = pd.merge(admissions, pivot_omr, how="left", on="subject_id")
df_helper.drop_columns(
    base_df,[
        "admit_provider_id",
        "discharge_location",
        "hospital_expire_flag",
        "edregtime",
        "edouttime",
])
filtered_df=df_helper.column_subset(
    patients,[
        "subject_id",
        "gender",
        "anchor_age"
])
base_df=pd.merge(filtered_df, base_df, on="subject_id", how="right")
#rename anchor_age to age
df_helper.rename_column(base_df, "anchor_age", "age")
base_df

In [ ]:
merged_diagnoses= pd.merge(diagnoses, diagnoses2, on=['icd_code', 'icd_version'], how='left')
merged_diagnoses.drop(["icd_code", "icd_version","seq_num"],axis=1,inplace=True)
#rename long_title column to diagnosis
merged_diagnoses.rename(columns={"long_title":"diagnosis"}, inplace=True)
#merge diagnoseswith the pivot_omr dataframe using the subject_id column
merged_diagnoses = pd.merge(merged_diagnoses, base_df, on="subject_id", how="right")
merged_diagnoses.drop("hadm_id_y", axis=1, inplace=True)
#rename hadm_id_x to hadm_id
merged_diagnoses.rename(columns={"hadm_id_x":"hadm_id"}, inplace=True)
merged_diagnoses


In [ ]:
death_df=admissions[["subject_id","hadm_id","deathtime"]].copy()
death_df["died"]=death_df["deathtime"].isna()==False
death_df.drop("deathtime",axis=1,inplace=True)
death_df.head()

In [ ]:
#Merge the death df with the merged diagnoses dataframe

diagnoses_death_df=pd.merge(merged_diagnoses, death_df, on=["subject_id","hadm_id"], how="left")
diagnoses_death_df["died"].fillna(False, inplace=True)
diagnoses_death_df.drop("deathtime",axis=1, inplace=True)
diagnoses_death_df

In [ ]:
# merging the filtered_df dataframe with the diagnoses_death_df dataframe using the subject_id column as the key. 
filtered_df=patients[["subject_id","anchor_age"]]
diagnoses_death_df=pd.merge(filtered_df, diagnoses_death_df, on="subject_id", how="right")
#rename anchor_age to age

diagnoses_death_df.rename(columns={"anchor_age":"age"}, inplace=True)
diagnoses_death_df

In [ ]:

length_of_stay_df =diagnoses_death_df.copy()

length_of_stay_df["admittime"]=pd.to_datetime(length_of_stay_df["admittime"])
length_of_stay_df["dischtime"]=pd.to_datetime(length_of_stay_df["dischtime"])

#calculate the length of stay in days

length_of_stay_df["length_of_stay"]=length_of_stay_df["dischtime"]-length_of_stay_df["admittime"]
length_of_stay_df["length_of_stay"]=length_of_stay_df["length_of_stay"].dt.days

length_of_stay_df



In [ ]:
#merging specific features from the prescriptions csv
prescriptions_df=length_of_stay_df.copy()
filtered_readmission_df =prescriptions[["subject_id","hadm_id","drug_type","drug"]]
prescriptions_df=pd.merge(filtered_readmission_df, prescriptions_df, on=["subject_id","hadm_id"],how="left")
#prescriptions.drop("gender_y", axis=1, inplace=True)
#rename gender_x to gender
prescriptions_df.rename(columns={"gender_x":"gender"}, inplace=True)
#dropping duplicates from the readmission_df
prescriptions_df.drop_duplicates(inplace=True)
prescriptions_df


In [ ]:
#Useful for TimeSeries
df=admissions.copy()
df.admittime = pd.to_datetime(df.admittime)   
df.dischtime = pd.to_datetime(df.dischtime)

df1 = df.groupby([df.admittime.dt.year, df.admittime.dt.month]).count().rename_axis(['year', 'month'])['admittime'].reset_index(name='Entries')

df1

In [ ]:
admissions.duplicated().value_counts()

In [ ]:
#Calculating the number of readmissions for each subject_id using a readmission threshold of 30 days
readmissions_df= admissions[["subject_id","hadm_id","admittime","dischtime"]].copy()


readmissions_df['admittime'] = pd.to_datetime(readmissions_df['admittime'])
readmissions_df['dischtime'] = pd.to_datetime(readmissions_df['dischtime'])

# Assuming df is your dataframe and readmission_threshold is defined
readmission_threshold = pd.Timedelta(days=30)

# Sort by subject_id, admittime
readmissions_df = readmissions_df.sort_values(by=['subject_id', 'admittime'])

# Calculate the time to next admission and determine if it's a readmission
readmissions_df['next_admittime'] = readmissions_df.groupby('subject_id')['admittime'].shift(-1)
readmissions_df['readmission_time'] = readmissions_df['next_admittime'] - readmissions_df['dischtime']
readmissions_df['is_readmission'] = (readmissions_df['readmission_time'] <= readmission_threshold) & (readmissions_df['readmission_time'] > pd.Timedelta(days=0))

# Calculate readmissions per patient
readmissions_df['readmissions'] = readmissions_df.groupby('subject_id')['is_readmission'].cumsum()

#merge readmission_df with base_df
base_copy = base_df.copy()
base_copy.drop(["admittime", "dischtime"], axis=1, inplace=True)
readmissions_df = pd.merge(readmissions_df, base_copy, on=['subject_id', 'hadm_id'], how='left')
readmissions_df

### Data Cleaning

In [ ]:
#Checking for duplicates and missing values from the prescriptions df

prescriptions_df.duplicated().sum()


In [ ]:
#missing values
prescriptions_df.isnull().sum()

In [ ]:
#drop missing values from the dataset

prescriptions_df.dropna(inplace=True)
prescriptions_df

In [ ]:
prescriptions_df.isnull().sum()

In [ ]:
cleaned_data= prescriptions_df.copy()
cleaned_data.head()

Dropping duplicate age column

In [ ]:
cleaned_data = cleaned_data.loc[:, ~cleaned_data.columns.duplicated(keep='first')]
cleaned_data.columns

In [ ]:
cleaned_data.to_csv("data/cleaned_data.csv", index=False)

## Exploratory Data Analysis


In [ ]:
import matplotlib.pyplot as plt  # For basic plotting
import seaborn as sns      # For enhanced statistical plots


In [ ]:
cleaned_data.head()

### a) Univariate Analysis
Univariate analysis involves examining the distribution and characteristics of individual variables(numerical features). We'll go through this step by step.




In [ ]:
# Function for univariate analysis
def univariate_analysis(df, column_name):
    # Summary statistics
    summary = cleaned_data[column_name].describe()
    print(f"Summary statistics for {column_name}:\n{summary}\n")

    # Histogram
    plt.figure(figsize=(8, 5))
    sns.histplot(cleaned_data[column_name], bins=10, kde=True)
    plt.title(f'Distribution of {column_name}')
    plt.xlabel(column_name)
    plt.ylabel('Frequency')
    plt.show()

    # Box plot to detect outliers
    plt.figure(figsize=(8, 5))
    sns.boxplot(x=cleaned_data[column_name])
    plt.title(f'Box Plot of {column_name}')
    plt.xlabel(column_name)
    plt.show()

#### 1. Distribution of Age

In [ ]:
univariate_analysis(cleaned_data, 'age')

* The age distribution is skewed left, indicating a higher proportion of older individuals with a mean age of 62.62 years and a standard deviation of 13.93 years. The data reveals a concentration of individuals between 53 and 72 years, with a peak around the 60-65 year age group, suggesting a predominantly older population.
* The box plot reveals a clear left-skewed distribution of age, characterized by a longer whisker towards the younger end of the age spectrum, indicating a larger proportion of older individuals within the dataset and a median age situated approximately at the 65-year mark.

#### 2. length_of_stay

In [ ]:
univariate_analysis(cleaned_data, 'length_of_stay')

* The distribution of length of stay is right-skewed, with a mean of 8.24 days and a median likely between 6-7 days (as indicated by the 50th percentile). This suggests that most patients have shorter stays, but there is a significant proportion of patients with longer stays, pulling the mean to the right. The distribution is characterized by a long tail, indicating the presence of outliers with exceptionally long lengths of stay.
* For the box plot,The presence of numerous outliers, representing patients with significantly longer stays, contributes to the elongated right tail of the distribution. This suggests that a small subset of patients account for a disproportionate amount of the overall length of stay.

### b) Bivariate Analysis

Used to investigate the relationship between two variables.

Lets start with a contingency table that shows the frequency distribution of variables

#### 1. Admission type vs Died


In [ ]:
# Crosstab for Admission Type vs Died
contingency_table = pd.crosstab(cleaned_data['admission_type'], cleaned_data['died'])
print(contingency_table)


In [ ]:
# Stacked bar plot for Admission Type vs Died
contingency_table.plot(kind='bar', stacked=True, figsize=(8, 5))
plt.title('Admission Type vs Died')
plt.xlabel('Admission Type')
plt.ylabel('Count')
plt.legend(title='Died')
plt.show()


 The chart shows that admission types like EW EMER. and OBSERVATION ADMIT have higher overall patient counts, with a noticeable proportion of deaths. In contrast, admission types like AMBULATORY OBSERVATION and DIRECT OBSERVATION have lower overall counts and fewer associated deaths. This suggests potential correlations between admission type and mortality rates, with certain admission types possibly indicating higher risks.

In [ ]:
#creating a function for Bivariate analysis
def bivariate_categorical_countplot(df, cat_var1, cat_var2):
    """
    Perform bivariate analysis for two categorical features using a count plot.
    
    Parameters:
    df (pd.DataFrame): The dataframe containing the data.
    cat_var1 (str): The first categorical variable.
    cat_var2 (str): The second categorical variable.
    """
    
    plt.figure(figsize=(10, 6))
    sns.countplot(data=cleaned_data, x=cat_var1, hue=cat_var2, palette='Set1')
    plt.title(f'{cat_var1} vs. {cat_var2}')
    plt.xlabel(cat_var1)
    plt.ylabel('Count')
    plt.legend(title=cat_var2)
    plt.show()


#### 2. Gender vs Died

In [ ]:

bivariate_categorical_countplot(cleaned_data, 'gender', 'died')

The bar chart illustrates a higher mortality rate among male patients compared to female patients.

#### 3. Marital vs Died

In [ ]:
bivariate_categorical_countplot(cleaned_data, 'marital_status', 'died')

The bar chart illustrates that married individuals have the highest count, followed by widowed, divorced, and single individuals, with a disproportionately higher number of deaths among single individuals relative to their population size.

#### 4. Age vs Length of stay

In [ ]:
cleaned_data.age

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(cleaned_data['age'], cleaned_data['length_of_stay'], alpha=0.5)
plt.title('Scatter Plot of Age vs. Length of Stay')
plt.xlabel('Age')
plt.ylabel('Length of Stay (days)')
plt.grid(True)
plt.show()


The scatter plot indicates a weak or no correlation between age and length of stay. There is a wide range of lengths of stay for each age group, suggesting that factors other than age primarily influence the duration of hospitalization.

#### 5. Length of stay vs Marital status

In [ ]:
plt.figure(figsize=(12, 6))
cleaned_data.boxplot(column='length_of_stay', by='marital_status', grid=False)
plt.title('Boxplot of Length of Stay by Marital Status')
plt.suptitle('')  # Suppresses the automatic title to make the plot cleaner
plt.xlabel('Marital Status')
plt.ylabel('Length of Stay (days)')
plt.show()


Married individuals tend to have shorter hospital stays compared to divorced, single, and widowed individuals. Additionally, there are outliers in most groups, indicating some patients with exceptionally long hospital stays regardless of marital status. There is also greater variability in length of stay among divorced and widowed individuals.